# 聊天数据优化
- 加入历史数据，选择最近5个回合记录
- 相同的问答进行采样

In [1]:
import json
import argparse
import pandas as pd
from tqdm import tqdm
import random

In [2]:
csv_path = '/home/nakai/project/aigc/llm/dataset/聊天内容/40v40聊天记录.csv'

In [3]:
df = pd.read_csv(csv_path, nrows=30000000)

/tmp/ipykernel_1152953/1954708365.py:1: DtypeWarning: Columns (0,1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, nrows=30000000)


In [4]:
conversations = {}
# 将消息组合成长文本
for _, row in tqdm(df.iterrows()):
    from_uid = row['src_id']
    to_uid = row['dest_id']
    msg = str(row['content']).strip()
    msg = str(from_uid) + '###' + str(row['content']).strip()

    # 将from_uid和to_uid按照字母顺序组合成一个元组，确保对话参与者的组合是唯一的
    participants = tuple(sorted([from_uid, to_uid]))

    if participants not in conversations:
        conversations[participants] = []
    if msg.find("{") >= 0:
        continue
    if len(conversations[participants]) > 0:
        last_msg = conversations[participants][-1]
        if last_msg.split('###')[0] == msg.split('###')[0]:
            conversations[participants][-1] = last_msg +","+msg.split('###')[-1]
        else:
            conversations[participants].append(msg)
    else:     
        conversations[participants].append(msg)ensure_ascii=

10536262it [07:13, 24278.54it/s]


In [5]:
# for idx in range(len(item)):
#     if idx == 0:
#         continue
#     print(item[idx -1], item[idx])
#     # last msg
#     history = []
#     last_idx = idx - 2
#     while len(history) < 5 and last_idx > 0:
#         history.insert(0, [item[last_idx - 1], item[last_idx]])
#         last_idx = last_idx - 2
#     print(history)

In [6]:
def clear_msg(msg):
    return msg.replace('\n', '').replace('"', '').replace('\'', '').replace('/', '').replace('\\', '').replace('^M', '')
# 构造单轮会话数据集，无提示词
'''
 {
    "instruction": "你还能做什么?",
    "input": "",
    "output": "我可以替你写代码，作文等",
    "history": []
  },
'''
cnt = 0

corpus = {}

for item in tqdm(conversations.values()):
    if len(item) <= 1:
        continue
    for idx in range(1, len(item)):
        cnt += 1
        ip = clear_msg(item[idx-1].split('###')[-1])
        ot = clear_msg(item[idx].split('###')[-1])
        # 往回找n条记录
        history = []
        last_idx = idx - 2
        while len(history) < 5 and last_idx > 0:
            history.insert(0, [clear_msg(item[last_idx - 1].split('###')[-1]), clear_msg(item[last_idx].split('###')[-1])])
            last_idx = last_idx - 2
        out = f"""{{
"instruction": "",
"input": "{ip}",
"output": "{ot}",
"history": {json.dumps(history, ensure_ascii=False)}
}}
"""
        key = f'{ip}_{ot}'
        if key not in corpus:
            corpus[key] = list()
        corpus[key].append(out)

    

100%|██████████| 168002/168002 [01:25<00:00, 1965.83it/s]  


In [7]:
k_cnt = list()
i_cnt = {}
for key in corpus:
    k_cnt.append((key, len(corpus[key])))
    k = key.split('_')[0]
    if k not in i_cnt:
        i_cnt[k] = 0
    i_cnt[k] += len(corpus[key])
    

In [8]:
sorted_k_cnt = sorted(k_cnt, key=lambda x: x[1], reverse=True)

sorted_i_cnt = sorted(list(i_cnt.items()),  key=lambda x: x[1], reverse=True)


In [9]:
sorted_k_cnt[:100]

[('早_早', 873),
 ('好吧_嗯', 590),
 ('嗯_嗯', 577),
 ('nan_nan', 557),
 ('哦_嗯', 520),
 ('好吧_嗯嗯', 323),
 ('晚安_晚安', 314),
 ('怎么了_没事', 304),
 ('嗯嗯_嗯', 296),
 ('好_嗯', 291),
 ('你好_你好', 284),
 ('嗯_嗯嗯', 273),
 ('哦哦_嗯', 244),
 ('晚安_早', 227),
 ('早啊_早', 208),
 ('哦哦_嗯嗯', 205),
 ('好的_嗯', 198),
 ('哦哦_你呢', 191),
 ('nan,nan_nan', 186),
 ('早安_早', 185),
 ('哦_嗯嗯', 181),
 ('😘_😘', 176),
 ('在吗_在', 174),
 ('nan_nan,nan', 172),
 ('哦_你呢', 165),
 ('咋了_没事', 165),
 ('？_？', 165),
 ('好_嗯嗯', 161),
 ('哈喽_哈喽', 157),
 ('😂_😂', 154),
 ('早呀_早', 153),
 ('真的吗_真的', 153),
 ('嗯_在干嘛呢', 149),
 ('真的假的_真的', 147),
 ('好的_嗯嗯', 143),
 ('嗯嗯_嗯嗯', 140),
 ('好吧_嗯呢', 137),
 ('没有_好吧', 137),
 ('早_早啊', 136),
 ('嗯呢_嗯', 136),
 ('哈哈哈_😂', 133),
 ('哈哈_嘿嘿', 131),
 ('哈哈_😂', 128),
 ('嗯_嗯呢', 125),
 ('好吧_嗯呐', 121),
 ('哈哈_笑什么', 113),
 ('嗯_好吧', 112),
 ('嗯_你呢', 109),
 ('🤔_🤔', 109),
 ('嘿嘿_嘻嘻', 106),
 ('早上好_早', 105),
 ('nan,nan_nan,nan', 105),
 ('是吗_是的', 103),
 ('你好呀_你好', 103),
 ('哈哈哈_😂😂', 102),
 ('吃饭了吗_吃了', 102),
 ('😂😂_😂😂', 99),
 ('哈哈哈_嘿嘿', 99),
 ('早_嗯', 98),
 (

In [10]:
sorted_i_cnt[:100]

[('嗯', 45652),
 ('嗯嗯', 24496),
 ('哈哈哈', 21043),
 ('哈哈', 19711),
 ('😂', 18364),
 ('没有', 13456),
 ('😂😂', 13426),
 ('好吧', 13317),
 ('好', 12100),
 ('？', 11533),
 ('哦', 11223),
 ('😂😂😂', 10425),
 ('对啊', 9221),
 ('是的', 8319),
 ('嘿嘿', 8283),
 ('哈哈哈哈', 8104),
 ('对', 7585),
 ('早', 7412),
 ('nan', 7372),
 ('怎么了', 7033),
 ('好的', 6807),
 ('哦哦', 6797),
 ('嗯呢', 6672),
 ('嘻嘻', 6135),
 ('对呀', 6036),
 ('没事', 5688),
 ('嗯呐', 5582),
 ('不是', 5439),
 ('🤔', 5364),
 ('🙄', 5183),
 ('嗯哼', 4947),
 ('不知道', 4845),
 ('哼', 4723),
 ('是啊', 4682),
 ('啊', 4151),
 ('晚安', 3811),
 ('😊', 3808),
 ('😅', 3806),
 ('咋了', 3755),
 ('没', 3747),
 ('你呢', 3746),
 ('不要', 3738),
 ('嗯？', 3681),
 ('😑', 3606),
 ('😀', 3602),
 ('没有啊', 3514),
 ('可以', 3500),
 ('为什么', 3485),
 ('额', 3478),
 ('真的', 3391),
 ('哈哈哈哈哈', 3255),
 ('你好', 3131),
 ('😏', 2987),
 ('啊？', 2835),
 ('不', 2819),
 ('☺', 2799),
 ('为啥', 2774),
 ('什么', 2763),
 ('在', 2499),
 ('🙈', 2482),
 ('唉', 2443),
 ('在干嘛呢', 2422),
 ('……', 2394),
 ('😘', 2349),
 ('有', 2327),
 ('？？？', 2301),
 ('怎么啦',

In [11]:
# filter nan
# sample by kn，input sample 大些， input sample 小一点

In [13]:
with open('/home/nakai/project/aigc/llm/LLaMA-Factory/data/chat_dataset_v2/chat_v2.json', mode='w') as tf:
    tf.write('[\n')
    cnt = 0
    for io in tqdm(corpus):
        if io.find('nan') >= 0:
            continue
        items = corpus[io]
        ip = io.split('_')[0]
        hsize = i_cnt[ip]
        if hsize > 1000:
            rh = random.randint(1, hsize)
            if rh > 0.1 * hsize:
                continue
        
        sub_list_length = min(30, len(items))
        sub_list = random.sample(items, sub_list_length)
        for out in sub_list:
            cnt += 1
            sep = ''
            if cnt > 1:
                sep = ','
            tf.write(f'{sep}{out}')
    tf.write(']')

100%|██████████| 5448039/5448039 [00:30<00:00, 176265.86it/s]


In [69]:
sorted_i_cnt

[('嗯', 45652),
 ('嗯嗯', 24496),
 ('哈哈哈', 21043),
 ('哈哈', 19711),
 ('😂', 18364),
 ('没有', 13455),
 ('😂😂', 13426),
 ('好吧', 13317),
 ('好', 12099),
 ('？', 11532),
 ('哦', 11221),
 ('😂😂😂', 10425),
 ('对啊', 9221),
 ('是的', 8319),
 ('嘿嘿', 8282),
 ('哈哈哈哈', 8104),
 ('对', 7585),
 ('早', 7412),
 ('nan', 7372),
 ('怎么了', 7032),
 ('好的', 6807),
 ('哦哦', 6797),
 ('嗯呢', 6672),
 ('嘻嘻', 6135),
 ('对呀', 6036),
 ('没事', 5688),
 ('嗯呐', 5582),
 ('不是', 5439),
 ('🤔', 5364),
 ('🙄', 5183),
 ('嗯哼', 4947),
 ('不知道', 4845),
 ('哼', 4723),
 ('是啊', 4682),
 ('啊', 4151),
 ('晚安', 3811),
 ('😊', 3808),
 ('😅', 3806),
 ('咋了', 3755),
 ('没', 3747),
 ('你呢', 3746),
 ('不要', 3738),
 ('嗯？', 3681),
 ('😑', 3606),
 ('😀', 3602),
 ('没有啊', 3514),
 ('可以', 3500),
 ('为什么', 3485),
 ('额', 3478),
 ('真的', 3391),
 ('哈哈哈哈哈', 3255),
 ('你好', 3131),
 ('😏', 2987),
 ('啊？', 2835),
 ('不', 2819),
 ('☺', 2799),
 ('为啥', 2774),
 ('什么', 2763),
 ('在', 2499),
 ('🙈', 2482),
 ('唉', 2443),
 ('在干嘛呢', 2422),
 ('……', 2393),
 ('😘', 2349),
 ('有', 2327),
 ('？？？', 2301),
 ('怎么啦',